In [10]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import timedelta
import itertools

In [11]:
# Load file
path = r"Stock_Closing_Prices_10.csv"
stock_data = pd.read_csv(path, parse_dates=['Date'], index_col='Date')

# Time period
start_period = '2020-01-02'
end_period = '2022-12-30'
stock_period = stock_data.loc[start_period:end_period].copy()

# Parameters
buy_fee_rate = 0.0049  # 0.49% buy fee
sell_fee_rate = 0.003  # 0.3% sell fee
initial_investment_amount = 100000
annual_risk_free_rate = 0.02  # 2% risk-free rate
trading_days_per_year = 252
daily_risk_free_rate = (1 + annual_risk_free_rate) ** (1 / trading_days_per_year) - 1

# Fill NaN
stock_data.ffill(inplace=True)
stock_data.bfill(inplace=True)

# Generate parameter combinations
hist_lag_values = range(252, 505, 126)  # Small test values
rebalancing_frequency_values = range(21, 253, 21)
risk_tolerance_values = np.arange(0.1, 1.1, 0.1)

# All combinations are shown as below. The above are small tests.
# hist_lag_values = range(252, 505, 21)  # [252, 273, ..., 504]
# rebalancing_frequency_values = range(21, 253, 21)  # [21, 42, ..., 252]
# risk_tolerance_values = np.arange(0.1, 1.1, 0.1)  # [0.1, 0.2, ..., 1.0]

parameter_combinations = [
    (hl, rf, rt)
    for hl, rf, rt in itertools.product(hist_lag_values, rebalancing_frequency_values, risk_tolerance_values)
    if hl >= rf
]

print(f"Total parameter combinations: {len(parameter_combinations)}")

Total parameter combinations: 360


In [12]:
portfolio_performance = []
weights_records = []
investment_records = []
shares_records = []
trade_records = []
hold_records = []

In [13]:
hold_matrix = pd.DataFrame(0.0, index=stock_data.index, columns=stock_data.columns)
investment_amount_series = pd.Series(initial_investment_amount, index=stock_data.index, dtype=float)
trade_matrix = pd.DataFrame(0.0, index=stock_data.index, columns=stock_data.columns)

In [14]:
def objective_function(weights, mean_returns, cov_matrix, risk_tolerance):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_risk = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_risk - risk_tolerance * portfolio_return

In [15]:
# Main loop for parameter combinations
for hist_lag, rebalancing_frequency, risk_tolerance in parameter_combinations:
    
    start_period_with_lag = (pd.to_datetime(start_period) - timedelta(days=hist_lag)).strftime('%Y-%m-%d')
    stock_data = stock_period.loc[start_period_with_lag:end_period].copy()
    
    print(f"Running combination: hist_lag={hist_lag}, rebalancing_frequency={rebalancing_frequency}, risk_tolerance={risk_tolerance}")
    rebalance_dates = stock_data.index[::rebalancing_frequency]
    investment_amount = initial_investment_amount

    for i, current_date in enumerate(rebalance_dates):
        start_date = current_date - timedelta(days=hist_lag)
        historical_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= current_date)].copy()
        historical_data.interpolate(method='linear', inplace=True)
        historical_data = historical_data.loc[:, historical_data.isnull().mean() < 0.2]

        if historical_data.empty or historical_data.shape[1] < hist_lag:
            print(f"Skipping {current_date} due to insufficient data.")
            continue

        ret = np.log(historical_data).diff().dropna(how='all', axis=1)
        mean_returns = ret.mean()
        cov_matrix = ret.cov()

        if len(mean_returns) == 0:
            print(f"No assets available for optimization at {current_date}. Skipping.")
            continue
        
        print(f"Optimization at {current_date}. Running.")
        
        initial_weights = np.ones(len(mean_returns)) / len(mean_returns)
        bounds = [(0, 1) for _ in range(len(mean_returns))]
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

        optimized = minimize(objective_function, initial_weights, args=(mean_returns, cov_matrix, risk_tolerance),method='SLSQP', bounds=bounds, constraints=constraints)

        if not optimized.success:
            print(f"Optimization failed for {current_date},use previous weights.")
            if i > 0:
                previous_weights = weights_records[-1]
                aligned_weights = previous_weights.reindex(mean_returns.index, fill_value=0).values
                weights = aligned_weights / aligned_weights.sum()
            else:
                weights = initial_weights
        else:
            weights = optimized.x

        portfolio_return = np.dot(weights, mean_returns)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - daily_risk_free_rate) / portfolio_risk * np.sqrt(trading_days_per_year)

        # For each stock: calculate investment amount and # of shares
        latest_prices = historical_data.iloc[-1]
        investment_per_stock = weights * investment_amount
        shares_per_stock = (investment_per_stock / latest_prices).round(2)

        # Calculate trade_matrix and hold_matrix
        if i == 0:
            previous_shares = pd.Series(0, index=historical_data.columns)
        else:
            previous_shares = hold_matrix.loc[rebalance_dates[i - 1]]
        trades = shares_per_stock - previous_shares
        trade_matrix.loc[current_date, trades.index] = trades

        # Update holding
        hold_matrix.loc[current_date, trades.index] = shares_per_stock

        # Calculate transaction fee and update investment amount with returns and fees
        sell_trades = trades[trades < 0]
        buy_trades = trades[trades > 0]
        sell_cost = (-sell_trades * latest_prices.loc[sell_trades.index] * sell_fee_rate).sum()
        buy_cost = (buy_trades * latest_prices.loc[buy_trades.index] * buy_fee_rate).sum()
        investment_amount -= (sell_cost + buy_cost)

        if i > 0:
            prev_date = rebalance_dates[i - 1]
            hold_matrix.loc[prev_date:current_date - timedelta(days=1)] = hold_matrix.loc[prev_date].values
            investment_amount_series.loc[rebalance_dates[i - 1]:current_date - timedelta(days=1)] = investment_amount_series.loc[rebalance_dates[i - 1]]
        investment_amount_series.loc[current_date] = investment_amount
        
                # if shares_per_stock is too small, then adjust it
        significant_shares_mask = (shares_per_stock * 100).astype(int) > 0
        if not significant_shares_mask.all():
            significant_weights = weights[significant_shares_mask]
            total_significant_weight = significant_weights.sum()
            adjusted_weights = significant_weights / total_significant_weight
            adjusted_investment_per_stock = adjusted_weights * investment_amount
            adjusted_shares_per_stock = (adjusted_investment_per_stock / latest_prices[significant_shares_mask] * 100).round(2)

            weights = adjusted_weights
            investment_per_stock = adjusted_investment_per_stock
            shares_per_stock = adjusted_shares_per_stock
            historical_data = historical_data.loc[:, significant_shares_mask]
            
        # portfolio details
        weights_records.append(pd.Series(weights, index=historical_data.columns, name=current_date))
        investment_records.append(pd.Series(investment_per_stock, index=historical_data.columns, name=current_date))
        shares_records.append(pd.Series(shares_per_stock, index=historical_data.columns, name=current_date))
        
        portfolio_performance.append({
            'Date': current_date,
            'Portfolio Return': portfolio_return,
            'Portfolio Risk': portfolio_risk,
            'Sharpe Ratio': sharpe_ratio,
            'hist_lag': hist_lag,
            'rebalancing_frequency': rebalancing_frequency,
            'risk_tolerance': risk_tolerance,
            'Portfolio Combination': shares_per_stock.to_dict()
        })

Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.1
No assets available for optimization at 2020-01-02 00:00:00. Skipping.
Optimization at 2020-02-03 00:00:00. Running.
Optimization at 2020-03-04 00:00:00. Running.
Optimization at 2020-04-02 00:00:00. Running.
Optimization at 2020-05-04 00:00:00. Running.
Optimization at 2020-06-03 00:00:00. Running.
Optimization at 2020-07-02 00:00:00. Running.
Optimization at 2020-08-03 00:00:00. Running.
Optimization at 2020-09-01 00:00:00. Running.
Optimization at 2020-10-01 00:00:00. Running.
Optimization at 2020-10-30 00:00:00. Running.
Optimization at 2020-12-01 00:00:00. Running.
Optimization at 2020-12-31 00:00:00. Running.
Optimization at 2021-02-02 00:00:00. Running.
Optimization at 2021-03-04 00:00:00. Running.
Optimization at 2021-04-05 00:00:00. Running.
Optimization at 2021-05-04 00:00:00. Running.
Optimization at 2021-06-03 00:00:00. Running.
Optimization at 2021-07-02 00:00:00. Running.
Optimization at 2021-

In [16]:
# For the last rebalancing date
for date in hold_matrix.loc[rebalance_dates[-1]:].index:
    hold_matrix.loc[date] = hold_matrix.loc[rebalance_dates[-1]].values
    investment_amount_series.loc[date] = investment_amount_series.loc[rebalance_dates[-1]]

In [17]:
# Convert results to DataFrame
portfolio_df = pd.DataFrame(portfolio_performance)
print(f"Portfolio performance records: {len(portfolio_df)}")

Portfolio performance records: 2060


In [22]:
# Generate summary DataFrame
summary_data = []
for hist_lag, rebalancing_frequency, risk_tolerance in parameter_combinations:
    combination_df = portfolio_df[
        (portfolio_df['hist_lag'] == hist_lag) &
        (portfolio_df['rebalancing_frequency'] == rebalancing_frequency) &
        (portfolio_df['risk_tolerance'] == risk_tolerance)
    ]

    if combination_df.empty:
        continue

    avg_return = combination_df['Portfolio Return'].mean()
    avg_volatility = combination_df['Portfolio Risk'].mean()
    avg_sharpe_ratio = combination_df['Sharpe Ratio'].mean()

    latest_portfolio_combination = combination_df['Portfolio Combination'].iloc[-1]

    last_investment_amount = investment_amount_series.iloc[-1]
    last_holdings = hold_matrix.iloc[-1].to_dict()
    
    summary_data.append({
        'hist_lag': hist_lag,
        'rebalancing_frequency': rebalancing_frequency,
        'risk_tolerance': risk_tolerance,
        'Return': avg_return,
        'Volatility': avg_volatility,
        'Sharpe Ratio': avg_sharpe_ratio,
        'Portfolio Combination': latest_portfolio_combination,
        'Final Investment Amount': last_investment_amount,  # Add last investment amount
        'Final Holdings': last_holdings
    })

summary_df = pd.DataFrame(summary_data)

# Sort by Volatility in descending order and calculate risk_score
summary_df.sort_values(by='Volatility', ascending=False, inplace=True)
summary_df['risk_score'] = np.linspace(1, 0.01, len(summary_df))
summary_df['risk_score'] = summary_df['risk_score'].round(2)

# Save results to CSV
summary_df.to_csv('summary_results_with_risk_score-1118-1258.csv', index=False)
print("Summary table saved.")

Summary table saved.


In [19]:
print(summary_df.head())

     hist_lag  rebalancing_frequency  risk_tolerance    Return  Volatility  \
159       378                     84             1.0  0.006256    0.033998   
89        252                    189             1.0  0.006184    0.033859   
88        252                    189             0.9  0.006116    0.033113   
87        252                    189             0.8  0.006074    0.032612   
158       378                     84             0.9  0.006110    0.032433   

     Sharpe Ratio                              Portfolio Combination  \
159      2.834474  {'CF': 15285.1, 'DVN': 23236.34, 'EQT': 61291....   
89       2.898715  {'APA': 41985.31, 'CF': 15539.51, 'DVN': 35318...   
88       2.921116  {'APA': 41304.03, 'CF': 16193.05, 'DVN': 35281...   
87       2.939898  {'APA': 40372.94, 'CF': 17005.78, 'DVN': 35141...   
158      2.903236  {'CF': 15735.38, 'DVN': 22127.28, 'EQT': 59487...   

     Final Investment Amount  \
159             98521.069461   
89              98521.069461   
88

In [21]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from datetime import timedelta

np.random.seed(1)  # For reproducibility

output_dir = 'backtest_charts'
os.makedirs(output_dir, exist_ok=True)

# Load historical stock data
file_path = 'Stock_Closing_Prices_10.csv' 
data = pd.read_csv(file_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)
data.index = data.index.strftime('%Y-%m-%d')


start_period = '2020-01-02'
end_period = '2022-12-30'
stock_period = stock_data.loc[start_period:end_period].copy()

start_period_with_lag = (pd.to_datetime(start_period) - timedelta(days=hist_lag)).strftime('%Y-%m-%d')
data = stock_period.loc[start_period_with_lag:end_period].copy()


# Define the stock list and date range
# stock_list = [ 'AAPL', 'GOOGL', 'AMZN', 'NFLX', 'TSLA',  'NVDA']
stock_list=data.columns.tolist()
date_range = data.index.unique()

# Parameters for combinations
hist_lag_values = range(252, 505, 126)  # Small test values
rebalancing_frequency_values = range(21, 253, 21)
risk_tolerance_values = np.arange(0.1, 1.1, 0.1)

# Generate parameter combinations
parameter_combinations = [
    (hl, rf, rt)
    for hl, rf, rt in itertools.product(hist_lag_values, rebalancing_frequency_values, risk_tolerance_values)
    if hl >= rf
]

print(f"Total parameter combinations: {len(parameter_combinations)}")

combination_results = {}

for hist_lag, rebalancing_frequency, risk_tolerance in parameter_combinations:
    print(f"Running combination: hist_lag={hist_lag}, rebalancing_frequency={rebalancing_frequency}, risk_tolerance={risk_tolerance}")
    
    # Create the holdings DataFrame
    holdings1 = pd.DataFrame(0.0, index=date_range, columns=stock_list + ['cash', 'fee', 'cumulative_fee'])
    holdings2 = pd.DataFrame(0.0, index=date_range, columns=stock_list + ['cash', 'fee', 'cumulative_fee'])

    # Initialize cash and cumulative fees
    holdings1.iloc[0, -3] = 100000.0  # Initial cash for trades1
    holdings2.iloc[0, -3] = 0  # Initial cash for trades2
    holdings1.iloc[0, :-3] = 0 # Initial holdings for trades1
    holdings2.iloc[0, :-3] = 100  # Initial holdings for trades2

    buy_fee_rate = 0.0049  # 0.49% buy fee
    sell_fee_rate = 0.003  # 0.3% sell fee

    # Extract the closing prices
    close_prices = data[stock_list]

    # Create trade decision matrices trades1 and trades2
    trades1 = pd.DataFrame(0, index=close_prices.index, columns=stock_list)
    trades2 = pd.DataFrame(0, index=close_prices.index, columns=stock_list)


    # Load the CSV file and set 'Date' as the index
    trades1 = trade_matrix
    trades2 = pd.DataFrame(np.random.choice([0, 0, 0], size=(len(date_range), len(stock_list))), 
                        index=date_range, 
                        columns=stock_list)


    # Process trades for trades1
    for date in date_range:
        # print(f'processing date:{date} ')
        if date == date_range[0]:
            holdings1.loc[date, 'cash'] = 100000.0  # Initial cash
            holdings1.loc[date, 'fee'] = 0.0
            holdings1.loc[date, 'cumulative_fee'] = 0.0
        else:
            previous_date = date_range[date_range.get_loc(date) - 1]  # Find previous date
            holdings1.loc[date, ['cash'] + stock_list + ['fee', 'cumulative_fee']] = holdings1.loc[previous_date, ['cash'] + stock_list + ['fee', 'cumulative_fee']]  # Copy previous data

        for stock in stock_list:
            trade = trades1.loc[date, stock]
            cost = trade * close_prices.loc[date, stock]
            
            # Handle buying
            if trade > 0:
                if holdings1.loc[date, 'cash'] >= (cost + abs(trade) * close_prices.loc[date, stock] * buy_fee_rate):  # Check if enough cash
                    fee = abs(trade) * close_prices.loc[date, stock] * buy_fee_rate
                    holdings1.loc[date, 'cash'] -= (cost + fee)  # Deduct total cost including fee
                    holdings1.loc[date, stock] += trade  # Update stock holdings
                    holdings1.loc[date, 'fee'] += fee  # Update fee


            # Handle selling
            elif trade < 0:
                if holdings1.loc[date, stock] + trade >= 0:  # Check if enough shares
                    holdings1.loc[date, stock] += trade
                    holdings1.loc[date, 'cash'] += abs(trade) * close_prices.loc[date, stock]
                    fee = abs(trade) * close_prices.loc[date, stock] * sell_fee_rate  # Calculate sell fee
                    holdings1.loc[date, 'cash'] -= fee  # Deduct fee from cash
                    holdings1.loc[date, 'fee'] += fee  # Update fee


    # Process trades for trades2 (similar to trades1)
    for date in date_range:
        if date == date_range[0]:
            holdings2.loc[date, 'cash'] = 100000.0  # Initial cash
            holdings2.loc[date, 'fee'] = 0.0
            holdings2.loc[date, 'cumulative_fee'] = 0.0
        else:
            previous_date = date_range[date_range.get_loc(date) - 1]  # Find previous date
            holdings2.loc[date, ['cash'] + stock_list + ['fee', 'cumulative_fee']] = holdings2.loc[previous_date, ['cash'] + stock_list + ['fee', 'cumulative_fee']]  # Copy previous data

        for stock in stock_list:
            trade = trades2.loc[date, stock]
            cost = trade * close_prices.loc[date, stock]

            # Handle buying
            if trade > 0:
                if holdings2.loc[date, 'cash'] >= (cost + abs(trade) * close_prices.loc[date, stock] * buy_fee_rate):  # Check if enough cash
                    fee = abs(trade) * close_prices.loc[date, stock] * buy_fee_rate
                    holdings2.loc[date, 'cash'] -= (cost + fee)  # Deduct total cost including fee
                    holdings2.loc[date, stock] += trade  # Update stock holdings
                    holdings2.loc[date, 'fee'] += fee  # Update fee


            # Handle selling
            elif trade < 0:
                if holdings2.loc[date, stock] + trade >= 0:  # Check if enough shares
                    holdings2.loc[date, stock] += trade
                    holdings2.loc[date, 'cash'] += abs(trade) * close_prices.loc[date, stock]
                    fee = abs(trade) * close_prices.loc[date, stock] * sell_fee_rate  # Calculate sell fee
                    holdings2.loc[date, 'cash'] -= fee  # Deduct fee from cash
                    holdings2.loc[date, 'fee'] += fee  # Update fee


    # Calculate total value for both trades
    holdings1['total_value'] = (holdings1[stock_list] * close_prices[stock_list]).sum(axis=1) + holdings1['cash']
    holdings2['total_value'] = (holdings2[stock_list] * close_prices[stock_list]).sum(axis=1) + holdings2['cash']

    initial_value = sum(holdings1.iloc[0][stock] * close_prices.loc[date_range[0], stock] for stock in stock_list) + holdings1.iloc[0]['cash']
    # Calculate daily returns for both trades
    holdings1['portfolio_value'] = holdings1['total_value'].shift(1).fillna(initial_value)
    holdings1['return'] = (holdings1['total_value'] / holdings1['portfolio_value']) - 1

    holdings2['portfolio_value'] = holdings2['total_value'].shift(1).fillna(initial_value)
    holdings2['return'] = (holdings2['total_value'] / holdings2['portfolio_value']) - 1

    # Calculate cumulative returns
    holdings1['cumulative_return'] = (1 + holdings1['return']).cumprod() - 1
    holdings2['cumulative_return'] = (1 + holdings2['return']).cumprod() - 1


    # print(holdings1['cumulative_return'].head())
    # Plotting
    plt.figure(figsize=(14, 7))

    # Plot total value for trades1
    plt.subplot(3, 1, 1)
    plt.plot(holdings1.index, holdings1['total_value'], label='Total Portfolio Value (Trades 1)', color='blue')
    plt.plot(holdings2.index, holdings2['total_value'], label='Total Portfolio Value (Trades 2)', color='orange')
    plt.title('Total Portfolio Value Over Time')
    plt.xlabel('Date')
    plt.ylabel('Total Value')
    plt.grid()
    plt.legend()

    # Plot cumulative returns for both trades
    plt.subplot(3, 1, 2)
    plt.plot(holdings1.index, holdings1['cumulative_return'], label='Cumulative Return (Trades 1)', color='blue')
    plt.plot(holdings2.index, holdings2['cumulative_return'], label='Cumulative Return (Trades 2)', color='orange')
    plt.title('Cumulative Return Over Time')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.grid()
    plt.legend()

    # Plot cumulative fees for both trades
    plt.subplot(3, 1, 3)
    plt.plot(holdings1.index, holdings1['fee'], label='Cumulative Fee (Trades 1)', color='green')
    plt.plot(holdings2.index, holdings2['fee'], label='Cumulative Fee (Trades 2)', color='red')
    plt.title('Cumulative Fees Over Time')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Fee')
    plt.grid()
    plt.legend()

    plt.tight_layout()
    
    # plt.show()
    
    file_name = f'backtest_{hist_lag}_{rebalancing_frequency}_{risk_tolerance:.2f}.png'
    plt.savefig(os.path.join(output_dir, file_name))
    plt.close()

Total parameter combinations: 360
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.1
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.2
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.30000000000000004
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.4
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.5
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.6
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.7000000000000001
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.8
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=0.9
Running combination: hist_lag=252, rebalancing_frequency=21, risk_tolerance=1.0
Running combination: hist_lag=252, rebalancing_frequency=42, risk_tolerance=0.1
Running combination: hist_lag=252, rebalancing_frequenc

In [ ]:
# Update cumulative fee for the current date

trading_days_per_year = 252
# Define the annual risk-free rate
annual_risk_free_rate = 2 / 100  # 2% annual risk-free rate

# For holdings1
# Calculate annual return based on cumulative return at the end
annual_return1 = (1 + holdings1['cumulative_return'].iloc[-1]) ** (trading_days_per_year / len(holdings1['cumulative_return'])) - 1
# Calculate excess annual return
excess_return1 = annual_return1 - annual_risk_free_rate

# Calculate the standard deviation of daily returns and annualize it
daily_return_std1 = holdings1['return'].std()
annualized_volatility1 = daily_return_std1 * np.sqrt(trading_days_per_year)

# Calculate the annualized Sharpe ratio
sharpe_ratio_trades1 = excess_return1 / annualized_volatility1


######################
# For holdings2
# Calculate annual return based on cumulative return at the end
annual_return2 = (1 + holdings2['cumulative_return'].iloc[-1]) ** (trading_days_per_year / len(holdings2['cumulative_return'])) - 1
# Calculate excess annual return
excess_return2 = annual_return2 - annual_risk_free_rate

# Calculate the standard deviation of daily returns and annualize it
daily_return_std2 = holdings2['return'].std()
annualized_volatility2 = daily_return_std2 * np.sqrt(trading_days_per_year)

# Calculate the annualized Sharpe ratio
sharpe_ratio_trades2 = excess_return2 / annualized_volatility2

# Display results for both trading strategies
print(f"--- Holdings1 ---")
print(f"Annual Return (Trades 1): {annual_return1:.4f}")
print(f"Excess Return (Trades 1): {excess_return1:.4f}")
print(f"Annualized Volatility (Trades 1): {annualized_volatility1:.4f}")
print(f"Sharpe Ratio (Trades 1): {sharpe_ratio_trades1:.4f}")

print(f"\n--- Holdings2 ---")
print(f"Annual Return (Trades 2): {annual_return2:.4f}")
print(f"Excess Return (Trades 2): {excess_return2:.4f}")
print(f"Annualized Volatility (Trades 2): {annualized_volatility2:.4f}")
print(f"Sharpe Ratio (Trades 2): {sharpe_ratio_trades2:.4f}")
######################

# Calculate cash proportion for both portfolios
holdings1['cash_proportion'] = holdings1['cash'] / holdings1['total_value']
holdings2['cash_proportion'] = holdings2['cash'] / holdings2['total_value']

# Plot stock positions and cash proportion over time for holdings1
plt.figure(figsize=(14, 10))

# Plot stock positions for holdings1
plt.subplot(2, 1, 1)
for stock in stock_list:
    plt.plot(holdings1.index, holdings1[stock], label=f'{stock} Position')
plt.title("Stock Positions Over Time (Holdings1)")
plt.xlabel("Date")
plt.ylabel("Number of Shares Held")
# plt.legend()
plt.grid(True)

# Plot cash proportion for holdings1
plt.subplot(2, 1, 2)
plt.plot(holdings1.index, holdings1['cash_proportion'], label='Cash Proportion', color='purple')
plt.title("Cash Proportion Over Time (Holdings1)")
plt.xlabel("Date")
plt.ylabel("Cash Proportion")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Plot stock positions and cash proportion over time for holdings2
plt.figure(figsize=(14, 10))

# Plot stock positions for holdings2
plt.subplot(2, 1, 1)
for stock in stock_list:
    plt.plot(holdings2.index, holdings2[stock], label=f'{stock} Position')
plt.title("Stock Positions Over Time (Holdings2)")
plt.xlabel("Date")
plt.ylabel("Number of Shares Held")
# plt.legend()
plt.grid(True)

# Plot cash proportion for holdings2
plt.subplot(2, 1, 2)
plt.plot(holdings2.index, holdings2['cash_proportion'], label='Cash Proportion', color='purple')
plt.title("Cash Proportion Over Time (Holdings2)")
plt.xlabel("Date")
plt.ylabel("Cash Proportion")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()